### Ensuring Consistency Across Training & Inference Datasets: Feature Scaling
**Question**: Load a dataset (e.g., Boston Housing) and perform feature scaling. Ensure the
same scaling is applied during model inference with new data.

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import unittest
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress warnings from deprecated load_boston

def preprocess_features(X):
    """Clean and validate features before scaling."""
    X = X.copy()
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    if X.isnull().values.any():
        print("Warning: Missing or infinite values found. Dropping affected rows.")
        X.dropna(inplace=True)
    return X

def scale_features(X_train, X_test):
    """Apply StandardScaler to training data and transform test data with same scaler."""
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

def main():
    # Load and preprocess data
    boston = load_boston()
    X = pd.DataFrame(boston.data, columns=boston.feature_names)
    y = boston.target

    X = preprocess_features(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling
    X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)

    # Save the scaler
    joblib.dump(scaler, "boston_scaler.pkl")

    print("Scaling complete. Scaler saved to 'boston_scaler.pkl'.")
    print(f"X_train_scaled shape: {X_train_scaled.shape}, X_test_scaled shape: {X_test_scaled.shape}")

# --- Unit Tests ---
class TestScaler(unittest.TestCase):

    def test_standard_scaling(self):
        """Test if StandardScaler gives mean ~0 and std ~1"""
        data = np.array([[1, 2], [3, 4], [5, 6]])
        scaler = StandardScaler()
        scaled = scaler.fit_transform(data)
        self.assertTrue(np.allclose(np.mean(scaled, axis=0), [0, 0], atol=1e-7))
        self.assertTrue(np.allclose(np.std(scaled, axis=0), [1, 1], atol=1e-7))

    def test_shape_consistency(self):
        """Check that scaled output has same shape as input"""
        data = np.random.rand(10, 4)
        scaler = StandardScaler()
        scaled = scaler.fit_transform(data)
        self.assertEqual(data.shape, scaled.shape)

if __name__ == "__main__":
    main()
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: